<a href="https://colab.research.google.com/github/FelipeTufaile/NetworkClustering/blob/main/NetworkClustering/Notebooks/20230615_NetworkClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import sys

In [ ]:
def check_connection(clusterA, clusterB):
  """
  This functions receives two lists of elements and returns a boolean (True or False) indicating wether or not the two lists have at leat one element in common.
  The function also return a list of unique elements contained in the union of two lists.

  Args:
      clusterA (list): A list of elements.
      clusterB (list): A list of elements.

  Returns:
      tuple: A tuple holding the a boolean (True or False) indicating wether or not the clusters are connected.
  """
  # Connecting clusters
  connection = clusterA + clusterB

  # Removing duplicates in the 'connection'
  net_connection = list(set(connection))

  # Return True if clusterA and clusterB have one element in common and False otherwise
  connected_clusters = len(net_connection) < len(connection)

  # Return the 'connected_clusters' boolean and the net_connection
  return (connected_clusters, net_connection)

In [ ]:
check_connection(clusterA=['Chair', 'Table'], clusterB=['Penci', 'Eraser'])

(False, ['Table', 'Chair', 'Penci', 'Eraser'])

In [ ]:
check_connection(clusterA=['Chair', 'Table'], clusterB=['Penci', 'Eraser', 'Chair'])

(True, ['Table', 'Chair', 'Penci', 'Eraser'])

In [ ]:
def find_connections(cluster, network):
  """
  This function receives a cluster (list of elements) and a network (list of lists of elements - sub-clusters) and check if there a connection between the
  reference cluster and all sub-clusters in the network. The functin then return an updated version of the reference cluster and network along with a boolean
  indicating whether or not the network was updated. In the case the network was not updated the boolean will be set to False and the updated versions of the
  cluster and network will equal to the original version of these lists.

  Args:
      cluster (list): A list of elements.
      network (list): A list of lists.

  Returns:
      tuple: A tuple holding the updated reference cluster, the updated network and a boolean (True or False) indicating wether or not there was an update in the network.
  """
  # Defining a list to hold the new work... after removing all sub-clusters connected to the reference cluster
  updated_network = []

  # Defining a list to hold the updated reference cluster
  updated_cluster = cluster.copy()

  # Iterating through all network of sub-clusters
  for sub_cluster in network:

    # Checking connection between the updated_cluster and the sub_cluster
    connection, net_cluster = check_connection(clusterA=updated_cluster, clusterB=sub_cluster)

    # Updating the 'updated_cluster' in case a connection is verified
    if(connection):
      updated_cluster = net_cluster

    # Returning the sub_cluster instance to the updated_network in case no connection was verified
    else:
      updated_network.append(sub_cluster)

  # Flagging wether or not the network was updated
  updated = len(updated_network) < len(network)

  # Return the 'updated_cluster', the 'updated_network' and a boolean indicating wether or not the network was updated
  return (updated_cluster, updated_network, updated)

In [ ]:
find_connections(cluster=['Chair', 'Table'], network=[['Penci', 'Eraser', 'Chair'], ['Stapler', 'Pencil Sharpner']])

(['Table', 'Chair', 'Penci', 'Eraser'], [['Stapler', 'Pencil Sharpner']], True)

In [ ]:
def print_progress(current_time, start_time, n_clusters, n_subclusters, n_network):
  """
  This function receives the current time, the start time (when the network clustering process actually started), the number of clusters found in the network,
  as for the current time, the number of remaining sub-clusters in the network and the original size of the network. The function then prints a log of the
  status of the clustering process.

  Args:
      current_time (datetime): A datetime object informing the current date and time.
      start_time (datetime): A datetime object informing the date and time when the clustering process actually started.
      n_clusters (int): An integer indicating the number of clusters formed as for the current_time.
      n_subclusters (int): An integer indicating the number of sub-clusters remaining in the network as for the current_time.
      n_subclusters (int): An integer indicating the number of sub-clusters remaining in the network as for the current_time.

  Returns:
      The function prints a log of the status of the clustering process.
  """
  # Calculating the total run time as for the current_time
  run_time = current_time - start_time

  # Calculating the minutes part of the run_time
  run_time_m = int(run_time // 60)

  # Calculating the seconds part of the run_time
  run_time_s = int(run_time - run_time_m)

  # Calculating remaining time
  rem_time = run_time*n_subclusters / (n_network - n_subclusters)

  # Calculating the minutes part of the rem_time
  rem_time_m = int(rem_time // 60)

  # Calculating the seconds part of the rem_time
  rem_time_s = int(rem_time - rem_time_m)

  # Creating response string
  response = "Run time: {} min {} s | Remaining time: {} min {} s | Clusters found: {} | Remaining sub-clusters: {} | Total sub-clusters: {}"

  # Plot status
  sys.stdout.write('\r')
  sys.stdout.write(response.format(run_time_m, run_time_s, rem_time_m, rem_time_s, n_clusters, n_subclusters, n_network))
  sys.stdout.flush()

In [ ]:
class NetworkClustering:
  """
  This class receives a network of sub-cluster and regroup these cluster based on commun elements (relationships) between clusters.
  Ex.:
    input_network = [['Person 1', 'Person 2'], ['Person 3', 'Person 4'], ['Person 1', 'Person 5', 'Person 6']]
    output_network = [['Person 1', 'Person 2', 'Person 5', 'Person 6'], ['Person 3', 'Person 4']]

  Args:
      input_network (list): A list of clusters (lists) representing the orginal un-clustered network.

  Returns:
      output_network (list): A list of clusters (lists) representing the clustered network.
  """

  def __init__(self, network):
    """
    Initializes the NetworkClustering class by attaching a network to the class.

    Args:
        input_network (list): A list of clusters (lists) representing the orginal un-clustered network.

    """
    # Defining a mutable network variable
    self.input_network = network

    # Initializing clusters list
    self.output_network = []


  def run_clustering(self, ):
    """
    Runs the clustering process and returns an updated (clustered) version of the input_ntework.

    Returns:
        output_network (list):  A list of clusters (lists) representing the clustered network.

    """
    # Creating an alias variable network from the given network
    # This alias will change along the clustering process as new groups are formed
    network = self.input_network.copy()

    # Setting start time
    start_time = time.time()

    # Run until there is a valid link in network
    while(len(network) > 1):

      # Initializing update and group variables
      updated, cluster = True, network[0]

      # Run while there is no update or it is the first run
      while (updated):

        # Clustering sub-clusters
        cluster, network, updated = find_connections(cluster=cluster, network=network)

      # Add the new cluster to the clusters list
      self.output_network.append(cluster)

      # Printing progress
      print_progress(current_time=time.time(), start_time=start_time, n_clusters=len(self.output_network), n_subclusters=len(network), n_network=len(self.input_network))

    # Add any remaining sub-cluster in the network as an individual cluster of clusters
    if(len(network) == 1):

      # Add the new cluster to the clusters list
      self.output_network.append(network[0])

      # Updating the network alias
      network = []

      # Printing progress
      print_progress(current_time=time.time(), start_time=start_time, n_clusters=len(self.output_network), n_subclusters=len(network), n_network=len(self.input_network))

    else:
      pass

    # Return the object
    return self

# Sample case

In [ ]:
# Defining a input network
input_network = [
    ['Person 01', 'Person 02', 'Client id 1A', 'Client id 2A', 'Company AA'],                 # sub-cluster 1
    ['Person 03', 'Client id 3B', 'Company BB'],                                              # sub-cluster 2
    ['Person 04', 'Person 05', 'Client id 4C', 'Client id 5C', 'Company CC'],                 # sub-cluster 3
    ['Person 06', 'Person 03', 'Client id 6D', 'Client id 3D', 'Company DD'],                 # sub-cluster 4
    ['Person 07', 'Client id 7E', 'Company EE'],                                              # sub-cluster 5
    ['Person 08', 'Client id 8E', 'Company EE'],                                              # sub-cluster 6
    ['Person 09', 'Client id 9E', 'Company EE'],                                              # sub-cluster 7
]

In [ ]:
# Defining a network clustering object
cn = NetworkClustering(network=input_network)

In [ ]:
# Visualizing the input network
cn.input_network

[['Person 01', 'Person 02', 'Client id 1A', 'Client id 2A', 'Company AA'],
 ['Person 03', 'Client id 3B', 'Company BB'],
 ['Person 04', 'Person 05', 'Client id 4C', 'Client id 5C', 'Company CC'],
 ['Person 06', 'Person 03', 'Client id 6D', 'Client id 3D', 'Company DD'],
 ['Person 07', 'Client id 7E', 'Company EE'],
 ['Person 08', 'Client id 8E', 'Company EE'],
 ['Person 09', 'Client id 9E', 'Company EE']]

In [ ]:
# Visualizing the output network
cn.output_network

[]

In [ ]:
# Running clustering
cn_run = cn.run_clustering()

Run time: 0 min 0 s | Remaining time: 0 min 0 s | Clusters found: 4 | Remaining sub-clusters: 0 | Total sub-clusters: 7

In [ ]:
# Visualizing the output network
cn.output_network

[['Person 02', 'Client id 2A', 'Company AA', 'Person 01', 'Client id 1A'],
 ['Company DD',
  'Client id 3D',
  'Person 06',
  'Company BB',
  'Client id 3B',
  'Client id 6D',
  'Person 03'],
 ['Company CC', 'Person 04', 'Client id 4C', 'Person 05', 'Client id 5C'],
 ['Company EE',
  'Person 09',
  'Client id 8E',
  'Client id 9E',
  'Client id 7E',
  'Person 07',
  'Person 08']]

In [3]:
!pip install git+https://github.com/FelipeTufaile/NetworkClustering.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/FelipeTufaile/NetworkClustering.git to /tmp/pip-req-build-_sxiw1ef
  Running command git clone --filter=blob:none --quiet https://github.com/FelipeTufaile/NetworkClustering.git /tmp/pip-req-build-_sxiw1ef
  Resolved https://github.com/FelipeTufaile/NetworkClustering.git to commit a7c457154c5ff6326f05804662f24ff8b93d9308
  Preparing metadata (setup.py) ... done
  Using cached times-0.7-py2.py3-none-any.whl (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
  Created wheel for NetworkClustering: filename=NetworkClustering-0.1.0-py3-none-any.whl size=3736 sha256=bf2624869349a18459c3cd2b040568395ca9b189f8f55a123cfb58b2e17fe3ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_pu6qow/wheels/a5/6c/64/4395d9073430d4e7a854fe3fda09213b9bf0b1859f3556c420
Successfully built NetworkClustering


In [4]:
from NetworkClustering.functions import NetworkClustering

In [6]:
# Defining a input network
input_network = [
    ['Person 01', 'Person 02', 'Client id 1A', 'Client id 2A', 'Company AA'],                 # sub-cluster 1
    ['Person 03', 'Client id 3B', 'Company BB'],                                              # sub-cluster 2
    ['Person 04', 'Person 05', 'Client id 4C', 'Client id 5C', 'Company CC'],                 # sub-cluster 3
    ['Person 06', 'Person 03', 'Client id 6D', 'Client id 3D', 'Company DD'],                 # sub-cluster 4
    ['Person 07', 'Client id 7E', 'Company EE'],                                              # sub-cluster 5
    ['Person 08', 'Client id 8E', 'Company EE'],                                              # sub-cluster 6
    ['Person 09', 'Client id 9E', 'Company EE'],                                              # sub-cluster 7
]

In [7]:
# Defining a network clustering object
cn = NetworkClustering(network=input_network)

In [8]:
# Running clustering
cn_run = cn.run_clustering()

Run time: 0 min 0 s | Remaining time: 0 min 0 s | Clusters found: 4 | Remaining sub-clusters: 0 | Total sub-clusters: 7